<img align="right" src="../../additional_data/banner_siegel.png" style="width:1000px;">

# Advanced Xarray

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Products used**: `Sentinel-2`
* **Prerequisites**:  Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_jupyter_introduction.ipynb)
    * The basic structure of the eo2cube [satellite datasets](02_eo2cube.ipynb)
    * How to browse through the available [products and measurements](03_products_and_measurements.ipynb) of the eo2cube datacube 
    * How to [load data from the eo2cube datacube](04_loading_data_and_basic_xarray.ipynb) 

## Background

The Python library `xarray` simplifies working with labelled multi-dimension arrays. The library introduces labels in the forms of dimensions, coordinates and attributes on top of `numpy` arrays. This structure allows easier and more effective handling of remote sensing raster data in a Python environment. Therefore, it is essential to fully understand the structure of an `xarray`. A first introduction into the usage of `xarray` within the eo2cube environment was given in ["04_loading_data_and_basic_xarray"](04_loading_data_and_basic_xarray.ipynb). This notebook builds on this gained knowledge and attempts to give a deeper understanding of the `xarray` data structure of raster data. If you are interested in learning more about the structures of the original `xarray`, have a look at this [**"introduction to xarray" notebook**](intro_to_xarray.ipynb) within the "intro_to_python" directory.
To get more information about the `xarray` package, visit the [offical documentation website](http://xarray.pydata.org/en/stable/).

## Description

This notebook introduces users to the `xarray` library within the datacube environment. It aims to deepen the understanding of the `xarray` structure as a container for remote sensing raster data. Also it introduces useful `xarray` functions to effectivly work with raster data in the eo2cube environment. Within this notebook the following topics are covered:

* Application of built-in `xarray` functions for analyzing raster data

***

## Setting up
### Load packages

The `datacube` package is required to query the eo2cube datacube database and load the requested data. The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data. The `xarray` and `numpy` package are needed for the different methods and analysis steps within this notebook. 

In [1]:
import datacube
from odc.ui import with_ui_cbk
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

dc = datacube.Datacube(app = "nb_understand_ndArrays")

### Datacube connection and load data

First we connect to the datacube and load an example dataset from the eo2cube. For this we will use the `s2_l2a_namibia` product. `ds` contains scenes of the `s2_l2a_namibia` product from January 2020 to March 2020.

In [2]:
# Load Data Product
ds = dc.load(product = "s2_l2a_bavaria",
             measurements = ["blue", "green", "red"],
             x= (9.80, 10.1),
             y= (49.70, 49.85),
             time = ("2020-10-01", "2020-12-31"),
             group_by = "solar_day",
            progress_cbk=with_ui_cbk())

ds

<xarray.Dataset>
Dimensions:      (time: 37, y: 1696, x: 2182)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 2090 2048 2006 2064 2088 ... 298 273 176 186
    green        (time, y, x) int16 2002 2018 1996 2004 2072 ... 397 462 236 229
    red          (time, y, x) int16 1800 1810 1810 1810 1864 ... 618 758 445 360
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

<a id='index_array3'></a>
## **Advanced Indexing**
### 1) Temporal Subset

In the earlier tutorial we introduced `isel()`and `sel()` for indexing data. For both methods a **slicing** operator exists. If the function `slice()` is passed onto the index function, the dataset can be sliced. 
The first example uses the slicing by position method to select the first five scenes in `data_1`. The start value is included and the stop value is excluded.

#### I. Using index number

In [19]:
#slice(0,5)
#ds.isel(time=slice(0,5)).time
#ds.isel(time = [0,1,2,3,4]).time

<xarray.DataArray 'time' (time: 5)>
array(['2020-01-02T09:07:17.000000000', '2020-01-04T08:57:22.000000000',
       '2020-01-07T09:07:17.000000000', '2020-01-09T08:57:21.000000000',
       '2020-01-12T09:07:16.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-01-02T09:07:17 ... 2020-01-12T09:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [18]:
ds.isel(time=slice(0,5))

<xarray.Dataset>
Dimensions:      (time: 5, x: 164, y: 82)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-02T09:07:17 ... 2020-01-12T09:...
  * y            (y) float64 7.28e+06 7.28e+06 7.28e+06 ... 7.28e+06 7.28e+06
  * x            (x) float64 1.751e+05 1.751e+05 ... 1.767e+05 1.768e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) int16 823 861 889 769 698 ... 852 802 774 753 713
    green        (time, y, x) int16 1120 1146 1190 1086 ... 1140 1126 1068 1036
    red          (time, y, x) int16 1560 1596 1634 1530 ... 1644 1614 1532 1448
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

This example uses the slicing by label method to select the scenes between "2020-01-02" and "2020-01-12". Note, that when using the `slice()` function with the `sel()` method, both start and stop value are included.

#### II. Using `datetime64` data

In [3]:
ds.sel(time=slice("2020-01-01", "2020-01-31")) # Namibia
#ds.sel(time=slice("01-01-2020", "01-31-2020"))
#print(ds.sel(time=slice("2019-12-01","2019-12-25"))) # Bavaria

#ds.sel(time=slice("2020-01-01", "2020-02-31")) # error
#ds.sel(time=slice("2018-01-01", "2018-01-31")) # wrong time frame

<xarray.Dataset>
Dimensions:      (time: 0, y: 1696, x: 2182)
Coordinates:
  * time         (time) datetime64[ns] 
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 
    green        (time, y, x) int16 
    red          (time, y, x) int16 
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

In [ ]:
#ds.sel(time=slice("2020-01-01", "2020-01-31")).time

#### III. Using other time dimensions

`xarray` also includes some useful features for the inspection of the time dimension. It allows to easily extract additional information from a dataset. The following code automatically groups the time dimension in seasons ("DJF", "MAM", JJA", "SON"). Since `ds` only contains scens from winter months, only the label "DJF" will appear. There are a lot of other `time` dimensions arguments, e.g. `month`, `week`, `weekday`, `dayofyear`.

In [25]:
#ds.time
#using special .dt accessor
#ds.time.dt

In [4]:
ds.time.dt.season

<xarray.DataArray 'season' (time: 37)>
array(['SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON',
       'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON',
       'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF'], dtype='<U3')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
    spatial_ref  int32 25832

It is also possible to extract the "day of year" for a time step.

In [5]:
ds.time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 37)>
array([275, 277, 280, 282, 285, 287, 290, 292, 295, 297, 300, 302, 305,
       307, 310, 312, 315, 317, 320, 322, 325, 327, 330, 332, 335, 337,
       340, 342, 345, 347, 350, 352, 355, 357, 360, 362, 365])
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
    spatial_ref  int32 25832

In [39]:
#ds.groupby('time.season').mean()

### 2) Spatial Subset
It is possible to index and **slice within the x and y dimensions**. The following example selects the value for each band of the pixel in the second colum of the raster and the fifth row of the raster (`x=2,y=5`)

In [6]:
ds.isel(x=2, y=5)
#ds.isel(x=[0,1,2], y=5)

<xarray.Dataset>
Dimensions:      (time: 37)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
    y            float64 5.522e+06
    x            float64 5.575e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time) int16 2120 8232 4176 5832 1314 ... 3518 5384 4152 913
    green        (time) int16 2090 7868 4364 5812 1454 ... 3172 5104 4232 772
    red          (time) int16 1878 7540 4520 5752 1350 ... 2824 5172 4248 888
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

### 3) Combining Temporal and Spatial Subset

Again, this method can be combined with the `slice()` operator to do a spatial subset of the dataset based on the position of the pixels. If you know the actual coordinate (x,y) value (extent) of the spatial subset area, use the `sel()` function.
Additionally, this subset can also be sliced in the time dimensions.

The following example subsets the `ds` by the spatial location of the pixels. Only the pixels from the first to the fifth column and the pixels from the first to the fifth row are included in the output. Also the scenes where filtered in the time dimension between the first and fifth time step.

In [7]:
ds2 = ds.isel(time=slice(0,5), x= slice(0,5), y=slice(0,5))
ds2

#ds2.time
#plt.scatter(ds2.x.values, ds2.y.values)

<xarray.Dataset>
Dimensions:      (time: 5, y: 5, x: 5)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-10-11T10:...
  * y            (y) float64 5.523e+06 5.523e+06 5.523e+06 5.523e+06 5.522e+06
  * x            (x) float64 5.575e+05 5.575e+05 5.575e+05 5.575e+05 5.576e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 2090 2048 2006 2064 ... 1372 1464 1442 1372
    green        (time, y, x) int16 2002 2018 1996 2004 ... 1446 1504 1478 1462
    red          (time, y, x) int16 1800 1810 1810 1810 ... 1358 1404 1410 1410
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

## **Data Manipulation & Statistics**

This notebook presents some basic built-in functions of the `xarray` library to manipulate and transform data in a `xarray.Dataset`. In this notebook only a fraction of the available `xarray` functions are presented. For a complete overview of all the available functions and tools of the `xarray` package please visit the [documentation website](http://xarray.pydata.org/en/stable/). The [notebook 07](07_basic_analysis.ipynb) will cover this topic with a focus on an application oriented remote sensing approach.

###  1) Statistical Operation

The simple built-in functions allow the user to do simple calculations with a `xarray.Dataset`.
The **basic math** built-in `xarray` functions are:
* `min()`, `max()`
* `mean()`, `median()`
* `sum()`
* `std()`

The following code demonstrates the easy use of the `max()` function to extract the maximum value of the red band in the `ds` dataset.

In [8]:
print(ds.red.max())

<xarray.DataArray 'red' ()>
array(17872, dtype=int16)
Coordinates:
    spatial_ref  int32 25832


To apply a function to every value of a specified dimension (e.g. to calculate the mean of every time step) the `dim` argument in the basic math function must be define with the dimension label.

This examples calculates the mean of the `red` band for each pixel (defined by the unique `x`, `y` combination) over every time step. The result is a data array which can be used for further time series visualization and analysis.

In [9]:
print(ds.red.mean(dim=["x", "y"]))

#ds.red.mean(dim=["x", "y"]).values
#plt.plot(ds.red.mean(dim=["x", "y"]).values)

<xarray.DataArray 'red' (time: 37)>
array([3513.75555791, 7428.63458664, 5251.54826907, 5599.97612055,
       4980.46660985, 7077.94765545, 6885.31980543, 1446.17029718,
       5874.8446474 , 3810.05474033, 6602.77405617, 4403.20711725,
       2608.1316977 , 2853.91329332,  589.39485883,  692.53116542,
       8571.55076375, 7339.14678631,  795.57128894, 3360.20685702,
       4533.52550348, 5090.9864022 , 7339.69299738, 6177.68353991,
        627.87020384, 7116.82303268, 7347.82443567, 7253.055501  ,
       6689.64509527, 3474.42465504, 4835.02113238,  815.20175471,
       4999.8348073 , 5293.99861268, 5414.84523622, 4124.4152354 ,
       1539.69305142])
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
    spatial_ref  int32 25832


This examples works the other way around. It calculates the standard deviation of every pixel (`x`, `y`) over all timesteps of the dataset `ds`.

In [10]:
print(ds.red.std(dim="time"))

<xarray.DataArray 'red' (y: 1696, x: 2182)>
array([[2935.3688693 , 2934.78223377, 2935.39077977, ..., 3086.2421816 ,
        3072.4680469 , 3087.85420801],
       [2913.88235944, 2924.28158409, 2932.14549823, ..., 3101.7422958 ,
        3105.73193377, 3129.91102402],
       [2841.07979709, 2859.70187574, 2870.72275051, ..., 3137.01263866,
        3174.08824522, 3230.37410263],
       ...,
       [2945.11318009, 2927.1953232 , 2920.9827081 , ..., 2912.34471191,
        2922.93660609, 2917.67834548],
       [2921.09469834, 2924.85850544, 2934.11201545, ..., 2913.70859122,
        2943.63554728, 2938.87457453],
       [2935.44860603, 2930.38739577, 2944.52430091, ..., 2900.87222243,
        2949.07657905, 2965.49355945]])
Coordinates:
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832


Remember, to access the raw `numpy` array that stores the values of the resulting `xarray.DataArrays`, the suffix `.values` is needed. This allows you to work with the "actual" data values.

In [11]:
print(ds.blue.sum(dim=["x","y"]).values)
#plt.plot(ds.blue.sum(dim=["x","y"]).values)

[15049484668 31065472625 18255077546 19803193192 19870405338 29864597492
 28751930718  5090045458 23932607112 11404212792 25471572650 15422698367
  9840664840 10148398798  1260808132  1437484670 38332284420 28689781204
  2094994147  9824580175 18928177200 20704759058 32224858168 25775773315
  1562694514 28765837252 26901640734 29167413227 27107091293 12781074047
 17910283565  2152610081 18223708382 20566396131 22910790864 15052032640
  5126215021]


### 2) Conditional Operation

Using conditional operation can be very helpful when we need to analyse satellite scenes or pixels lies within our interests. The `where()` function provides the option to **mask** a `xarray.Dataset` based on a logical condition. By default, the function converts all values that match the condition to NaN values. This is extremly useful when applied in combination with a binary mask to mask your data to the desired values. The argument `other` let´s you define a subset value for all values that match the condition (default is `nan`). The argument `drop` drops all values which do not match with the condition.
The following example masks the datatset `ds` to only the values which have a reflectance value of greater than 700 in the `red` band.

In [12]:
print(ds.where(ds.red > 700))
#print(ds.where(ds.red < 700))

<xarray.Dataset>
Dimensions:      (time: 37, y: 1696, x: 2182)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 2.09e+03 2.048e+03 2.006e+03 ... nan nan
    green        (time, y, x) float64 2.002e+03 2.018e+03 1.996e+03 ... nan nan
    red          (time, y, x) float64 1.8e+03 1.81e+03 1.81e+03 ... nan nan
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref


This code subsets all zeros in the red band of the dataset `ds` in the first time stamp with the new value -9999.

In [13]:
replace = ds.red.isel(time=0).where(ds.red != 0, other = -9999)
#replace.values.min()

The implemented `xarray` function `isin()` allows to **test each value** of `xarray.Dataset` or `xarray.DataArray` whether it is in the elements defined within the function. It returns a boolean array which can be used as a mask.
This example checks all the values of the `red` measurement if the value is in an array from 0 to 550.

In [14]:
mask_red = ds.red.isin(range(550))
print(mask_red)

#plt.imshow(mask_red) #error
#plt.imshow(mask_red.isel(time=3))

<xarray.DataArray 'red' (time: 37, y: 1696, x: 2182)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, .

The created mask can easily be combined with the `where()` function to filter the dataset based on the predefined mask. In this case the `ds` dataset is masked with previously defined mask `mask_red`, which was based on a logical test if values of the `red` band are within a certain range of values.

In [15]:
print(ds.where(mask_red)) #masking

<xarray.Dataset>
Dimensions:      (time: 37, y: 1696, x: 2182)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T10:37:04 ... 2020-12-30T10:...
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 nan nan nan nan ... nan nan 176.0 186.0
    green        (time, y, x) float64 nan nan nan nan ... nan nan 236.0 229.0
    red          (time, y, x) float64 nan nan nan nan ... nan nan 445.0 360.0
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref


### 3) Resampling
Working with time series data, resampling is necessary if we want the data product aligns with the temporal window are interested in.

 - **resample()**

The **`resample()` method** allows to summarise the `xarray.Dataset` to bigger or smaller chunks based on a dimension. It handels both upsampling and downsampling. The argument `time` needs to be defined like a datetime-like coordinate. In the following example we resample the `ds` dataset to a monthly time intervall (`time = "m"`) and than calculate the median value for every resample chunk. _(this process takes a little while to run)_

In [16]:
print(ds.resample(time='m').median())

<xarray.Dataset>
Dimensions:      (time: 3, y: 1696, x: 2182)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-31 2020-11-30 2020-12-31
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) float64 4.32e+03 4.336e+03 ... 4.29e+03 4.318e+03
    green        (time, y, x) float64 4.248e+03 4.436e+03 ... 4.546e+03 4.59e+03
    red          (time, y, x) float64 3.994e+03 4.232e+03 ... 4.858e+03


 - **groupby() method**

The **`groupby()` method** can also be used within the `xarray` library to *aggregate data over time*. Time aggregation arguments can be e.g. "time.year", "time.season", "time.month", "time.week", "time.day".
The code below groups the `ds` dataset into two groups by year. Therefore, a new "dimension" `year` is created. Then the median for each `year` is calculated. _(this process takes a little while to run)_

In [17]:
print(ds.groupby("time.year").median(dim="time"))

<xarray.Dataset>
Dimensions:      (y: 1696, x: 2182, year: 1)
Coordinates:
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
  * year         (year) int64 2020
Data variables:
    blue         (year, y, x) float64 4.204e+03 4.252e+03 ... 5.08e+03 5.052e+03
    green        (year, y, x) float64 3.638e+03 3.786e+03 ... 4.764e+03 4.72e+03
    red          (year, y, x) float64 3.608e+03 3.594e+03 ... 5.16e+03 5.132e+03


### 4) Interpolation
Interpolation is a common solution dealing with missing remote sensing data, either caused by coarse temporal resolution of the satellite, high cloud cover or bad quality of the scenes. For example, sometimes a scene of a specfic date is not available in the dataset. With the implemented `interp()` it is possible to **interpolate data** for predefined time steps. The function takes the next usable scene before and after the specified date and interpolates their values (default interpolation method is "linear") to build a new `xarray.Dataset`.

In this example, the `ds` dataset has missing scenes on the "2020-12-08". The `interp()` function builds a "new" scene based on an linear interpolation from the two measurments before and after the new time step.

In [19]:
print(ds.time)

<xarray.DataArray 'time' (time: 37)>
array(['2020-10-01T10:37:04.000000000', '2020-10-03T10:27:05.000000000',
       '2020-10-06T10:37:02.000000000', '2020-10-08T10:27:07.000000000',
       '2020-10-11T10:37:04.000000000', '2020-10-13T10:27:05.000000000',
       '2020-10-16T10:37:02.000000000', '2020-10-18T10:27:07.000000000',
       '2020-10-21T10:37:03.000000000', '2020-10-23T10:27:05.000000000',
       '2020-10-26T10:37:01.000000000', '2020-10-28T10:27:07.000000000',
       '2020-10-31T10:37:03.000000000', '2020-11-02T10:27:04.000000000',
       '2020-11-05T10:37:00.000000000', '2020-11-07T10:27:06.000000000',
       '2020-11-10T10:37:02.000000000', '2020-11-12T10:27:02.000000000',
       '2020-11-15T10:36:59.000000000', '2020-11-17T10:27:04.000000000',
       '2020-11-20T10:37:01.000000000', '2020-11-22T10:27:02.000000000',
       '2020-11-25T10:36:58.000000000', '2020-11-27T10:27:02.000000000',
       '2020-11-30T10:36:58.000000000', '2020-12-02T10:27:01.000000000',
       '2020-1

In [20]:
ds_interp = ds.interp(time=["2020-12-08"])
print(ds_interp)

<xarray.Dataset>
Dimensions:      (y: 1696, x: 2182, time: 1)
Coordinates:
  * y            (y) float64 5.523e+06 5.523e+06 ... 5.506e+06 5.506e+06
  * x            (x) float64 5.575e+05 5.575e+05 ... 5.793e+05 5.793e+05
    spatial_ref  int32 25832
  * time         (time) datetime64[ns] 2020-12-08
Data variables:
    blue         (time, y, x) float64 7.477e+03 7.661e+03 ... 5.031e+03
    green        (time, y, x) float64 6.909e+03 7.11e+03 ... 4.584e+03 4.614e+03
    red          (time, y, x) float64 6.626e+03 6.89e+03 ... 4.468e+03 4.546e+03
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref


The `merge()` function allows to **merge/join** `xarray.Datasets` or variables. By default the `merge()` function uses an "inner" join as merging operation. 
In our example the interpolated `xarray.Dataset` created above is merged to the `ds` dataset by using the default `merge()` function.

In [21]:
print(ds.merge(ds_interp).time)

<xarray.DataArray 'time' (time: 38)>
array(['2020-10-01T10:37:04.000000000', '2020-10-03T10:27:05.000000000',
       '2020-10-06T10:37:02.000000000', '2020-10-08T10:27:07.000000000',
       '2020-10-11T10:37:04.000000000', '2020-10-13T10:27:05.000000000',
       '2020-10-16T10:37:02.000000000', '2020-10-18T10:27:07.000000000',
       '2020-10-21T10:37:03.000000000', '2020-10-23T10:27:05.000000000',
       '2020-10-26T10:37:01.000000000', '2020-10-28T10:27:07.000000000',
       '2020-10-31T10:37:03.000000000', '2020-11-02T10:27:04.000000000',
       '2020-11-05T10:37:00.000000000', '2020-11-07T10:27:06.000000000',
       '2020-11-10T10:37:02.000000000', '2020-11-12T10:27:02.000000000',
       '2020-11-15T10:36:59.000000000', '2020-11-17T10:27:04.000000000',
       '2020-11-20T10:37:01.000000000', '2020-11-22T10:27:02.000000000',
       '2020-11-25T10:36:58.000000000', '2020-11-27T10:27:02.000000000',
       '2020-11-30T10:36:58.000000000', '2020-12-02T10:27:01.000000000',
       '2020-1

The `xarray` package contains a variety of other useful functions in addition to those shown here. There is a function for almost every operation needed in data analysis. For more information about the `xarray` package visit the [documentation website](http://xarray.pydata.org/en/stable/) or work through the [notebook]() in the "intro_to_python" folder.

The `xarray.Datasets` in the eo2cube datacube environment are a useful and effective structure for handeling remote sensing raster data. In this notebook you learned the basic structure and application methods of `xarray.Datasets`and `xarray.DataArrays`. However, it is very useful to not only work with the "raw" datasets and values. Sometimes it is necessary to get a visual overview of the data. The next [notebook 06](06_plotting.ipynb) will cover how to plot `xarray` raster data nicely and efficient. This is a very useful application, as it is often more convenient to visualize the raster data.

## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/01_jupyter_introduction.ipynb)
2. [eo2cube](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/02_eo2cube_introduction.ipynb)
3. [Loading Data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/03_data_lookup_and_loading.ipynb)
4. [Xarray I: Data Structure](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/04_xarrayI_data_structure.ipynb)
5. ***Xarray II: Index and Statistics (this notebook)***
6. [Plotting data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/06_plotting_basics.ipynb)
7. [Spatial analysis](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/07_basic_analysis.ipynb)
8. [Parallel processing with Dask](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/08_parallel_processing_with_dask.ipynb)

The additional notebooks are designed for users to build up both basic and advanced skills which are not covered by the beginner's guide. Self-motivated users can go through them according to their own needs. They act as complements for the guide:
<br>

1. [Python's file management tools](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/I_file_management.ipynb)
2. [Image Processing basics using NumPy and Matplotlib](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/II_numpy_image_processing.ipynb)
3. [Vector Processing](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/III_process_vector_data.ipynb)
4. [Advanced Plotting](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/IV_advanced_plotting.ipynb)

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks! </font>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** February 2021